<a href="https://colab.research.google.com/github/HwangHanJae/Recommendation_Study/blob/main/inflearn_recsys/CF_user_evaluation_tendency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 사용자의 평가경향을 고려한 CF

사용자마다 평점을 매기는 경향이 다를 수 있습니다.  
예를들어 어떤 사용자는 평점을 높게 주어 대부분의 평점이 4 ~ 5점이 분포되어 있을 수 있고  
어떤 사용자는 평점을 낮게 주어 2 ~ 3점으로 분포가 되어 있을 수 있습니다.  

두 사용자가 특정 영화에 대하여 동일하게 3점을 주었다고 하면 같은 의미로 해석할 수는 없을 것입니다.

사용자 기반 협업 필터링에서는 사용자들의 평가 경향을 고려하여 평점을 예측해야 더 정확한 평점 결과와 의미가 담길 것 입니다.

## 방법
1. 각 사용자의 평점평균을 계산
2. 평점편차를 계산 
-  평점편차 : 평점 - 해당 사용자의 평점 평균
3. 평점편차의 예측값을 계산
- 평가값 : 평점편차 * 유사도
4. 실제예측값 = 평가값 + 평점평균
- 평점편차를 구할 때 평점평균을 빼주었으니 다시 더해줌

# 데이터 읽기

무비렌즈의 유저의 정보(u.user) 읽기

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
#베이스 경로 설정
base = '/content/drive/MyDrive/RecoSys/Data'

# u.user 파일 경로 설정
u_user_path = os.path.join(base, 'u.user')

#필요한 컬럼 정의
u_cols = ['user_id','age','sex','occupation','zip_code']

#데이터 읽어오기
users = pd.read_csv(u_user_path, sep='|', names = u_cols, encoding='latin-1')
#users 데이터 프레임에 인덱스(user_id) 지정
users = users.set_index('user_id')

#상위 5개
users.head()

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


무비렌즈의 영화의 정보(u.item) 읽기

In [2]:
#u.item의 파일 경로 설정
u_item_path = os.path.join(base, 'u.item')

#필요한 컬럼 정의
i_cols = ['movie_id','title','release date','video release date','IMDB URL','unknown','Action',
          'Adventure','Animation','Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama',
          'Fantasy','Film-Noir','Horror','Musical', 'Mystery','Romance','Sci-Fi','Thriller','War','Western']

# 데이터 읽어오기
movies = pd.read_csv(u_item_path, sep='|',names =i_cols, encoding='latin-1')
# movies 데이터 프레임에 인덱스(movie_id) 지정
movies = movies.set_index('movie_id')

#상위 5개
movies.head()

,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


무비렌즈의 평점 정보(u.data)읽기

In [3]:
#u.data의 파일경로 지정
u_data_path = os.path.join(base, 'u.data')

#필요한 컬럼 정의
r_cols = ['user_id', 'movie_id','rating','timestamp']

#데이터 읽어오기
ratings = pd.read_csv(u_data_path, sep='\t',names = r_cols, encoding='latin-1')

#상위 5개
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


# 평가지표


In [4]:
from sklearn.metrics import mean_squared_error
def RMSE(y_true, y_pred):
  return np.sqrt(mean_squared_error(y_true, y_pred))

In [5]:
def score(model, neighbor_size=0):
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])

  y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])

  y_true = np.array(x_test['rating'])

  return RMSE(y_true, y_pred)

# 모델링

In [9]:
# 데이터 셋 만들기
x = ratings.copy()
y = ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25, stratify=y)

#sparse matrix 만들기
ratings_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

In [10]:
# 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity

matrix_dummy = ratings_matrix.copy().fillna(0)

user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity,
                               index=ratings_matrix.index,
                               columns=ratings_matrix.index)


# 사용자 평가 경향 고려

In [24]:
#사용자의 평점 평균을 구함
ratings_mean = ratings_matrix.mean(axis=1)
#평점 - 사용자의 평점 평균을 하여 평점 편차를 구함
ratings_bias = (ratings_matrix.T - ratings_mean).T


#함수
def CF_KNN_bias(user_id, movie_id, neighbor_size=0):
 #train set안에 movie_id가 존재한다면
  if movie_id in ratings_bias.columns:
    #해당 유저(user_id)와 다른 유저의 유사도를 구함 
    sim_scores = user_similarity[user_id].copy()
    #주어진 영화(movie_id)의 평점편차의 정보를 가져옴
    #어떤 유저가 평점을 했는지 안하였는지 확인할 수 있음
    movie_ratings = ratings_bias[movie_id].copy()
    #주어진 영화(movie_id)에 대하여 아직 평가를 하지 않은 유저정보(user_id)를 가져옴
    none_ratings_idx = movie_ratings[movie_ratings.isnull()].index
    #평가를 하지 않은 유저를 movie_ratings와 sim_scores에서 drop시킴
    movie_ratings = movie_ratings.drop(none_ratings_idx)
    sim_scores = sim_scores.drop(none_ratings_idx)
     #이웃을 지정하지 않았다면 사용자의 평가 경향을 고려하여 계산
    if neighbor_size == 0:
      #가중평균을 구해주고 사용자의 평점 평균을 더해줌
      prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
      prediction = prediction + ratings_mean[user_id]
    else:
      # 이웃을 지정하고 sim_score 크기가 1보다 크다면
      if len(sim_scores) > 1:
        # 이웃의 크기를 sim_score의 사이즈나 neighbor_size의 크기 중 작은것으로 선택
        # 주어진 이웃의 크기가 20이지만 sim_score의 크기가 10일 수 있음 이럴 경우 10을 선택
        neighbor_size = min(neighbor_size, len(sim_scores))
        sim_scores = np.array(sim_scores)
        movie_ratings = np.array(movie_ratings)
        # sim_score를 유사도값 기준으로 오름차순으로 정렬하고 index를 뽑음
        user_idx = np.argsort(sim_scores)
        # 이웃의 크기만큼 내림차순으로 sim_scores, movie_ratings를 구함
        # 이웃의 크기만큼의 피어그룹이 생성되고 그에 따른 영화의 평점편차 정보를 받을 수 있음
        sim_scores = sim_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]
        #가중평균을 구해주고 사용자의 평점 평균을 더해줌
        prediction = np.dot(sim_scores, movie_ratings) / np.sum(sim_scores)
        prediction = prediction + ratings_mean[user_id]
      
      else:
        # sim_score값이 0보다 작거나 같다면
        # 사용자가 평가한 영화들의 평점 평균을 지정
        prediction = ratings_mean[user_id]
  else:
    # 사용자가 평가한 영화들의 평점 평균을 지정
    prediction = ratings_mean[user_id]
  
  return prediction


In [25]:
score(CF_KNN_bias, 30)

0.9374199155363346

사용자 평가경향을 고려했을 때의 성능이 더 좋은 것으로 확인됩니다.

In [26]:
for neighbor_size  in [10, 20, 30, 40, 50, 60]:
  print(f'Neighbor size = {neighbor_size} : RMSE = {score(CF_KNN_bias, neighbor_size)}')

Neighbor size = 10 : RMSE = 0.9537553516701069
Neighbor size = 20 : RMSE = 0.9408382673196816
Neighbor size = 30 : RMSE = 0.9374199155363346
Neighbor size = 40 : RMSE = 0.9372436925625367
Neighbor size = 50 : RMSE = 0.9379824076002017
Neighbor size = 60 : RMSE = 0.938652986993242


최적의 이웃의 크기를 찾아서 결정할 수 있습니다. 40정도 가장 적합한 것으로 볼 수 있습니다.